In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:14 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-01-24 21:16:24--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.6’

postgresql-42.2.16. 100%[===================>] 979.38K  1.21MB/s    in 0.8s    

2021-01-24 21:16:25 (1.21 MB/s) - ‘postgresql-42.2.16.jar.6’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Home_Entertainment_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Home_Entertainment_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|  product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|     179886| RY01SAV7HZ8QO|B00NTI0CQ2|     667358431|Aketek 1080P LED ...|Home Entertainment|          4|            0|          0|   N|                Y|good enough for m...|not the best pict...| 2015-08-31|
|         US|   37293769|R1XX8SDGJ4MZ4L|B00BUCLVZU|     621695622|TiVo Mini with IR...|Home Entertainment|      

### Create DataFrames to match tables

In [5]:
from pyspark.sql.functions import to_date

In [6]:
# Create the customers_table DataFrame
customers_df = df.groupby("customer_id").agg({"customer_id":"count"}).withColumnRenamed("count(customer_id)", "customer_count")
customers_df.orderBy(customers_df.customer_count.desc()).show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   52340667|           111|
|   49837360|           103|
|   53075795|            49|
|   45664110|            43|
|   52641615|            42|
|   50861391|            35|
|    5478843|            31|
|   49067414|            30|
|   51117816|            30|
|   49557479|            29|
|   49266466|            29|
|   52801489|            27|
|   52765756|            27|
|   50273418|            26|
|   45418439|            26|
|   51712970|            26|
|   13844875|            26|
|   53037408|            26|
|   51787506|            26|
|   17957446|            25|
+-----------+--------------+
only showing top 20 rows



In [7]:
# Create the products_table DataFrame and drop duplicates. 
products_df = df.select(['product_id', 'product_title']).drop_duplicates(subset =['product_id'])
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|9967354976|Professional Retr...|
|B00000J025|RCA VR552 4-Head VCR|
|B00004YK3Q|Sylvania DVL500A ...|
|B00005NGRU|Toshiba 50H81 50-...|
|B00005U231|Sony KV36XBR450 3...|
|B000065UTY|Panasonic CT-27SX...|
|B00006CFLW|Hitachi 57XWX20B ...|
|B000093URS|Toshiba 65H93 65-...|
|B0000BZH5T|RCA 27F530T 27" T...|
|B00021Z98A|Panasonic DVD-LS5...|
|B0002IYMT0|Eveready 455 Carb...|
|B0002KQR3C|Samsung TX-P2675W...|
|B0002ZPTVS|Acoustic Wall Til...|
|B0002ZQGRY|Virgin Mobile Kyo...|
|B0006HJII2|Virtual Fx Conver...|
|B00078XIW2|Pioneer PDP4345HD...|
|B00080FPSE|Sony DVP-NC80V/S ...|
|B0009HGXT6|Lilliput 8" LCD M...|
|B0009KEYTO|Pioneer DVR-233S ...|
|B0009X47O8|Samsung DVD-VR325...|
+----------+--------------------+
only showing top 20 rows



In [8]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
review_id_df = df.select(['review_id','customer_id','product_id','product_parent', to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.drop_duplicates(subset=['review_id'])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| RY01SAV7HZ8QO|     179886|B00NTI0CQ2|     667358431| 2015-08-31|
|R1XX8SDGJ4MZ4L|   37293769|B00BUCLVZU|     621695622| 2015-08-31|
|R149Q3B5L33NN5|    8332121|B00RBX9D5W|     143071132| 2015-08-31|
|R2ZVD69Z6KPJ4O|   47054962|B00UJ3IULO|     567816707| 2015-08-31|
|R1DIKG2G33ZLNP|   23413911|B0037UCTXG|     909557698| 2015-08-31|
|R3L6FGKAW0EYFI|    4417771|B004N866SU|     414565179| 2015-08-31|
| RAO0QZH5VC6VI|   47900707|B00JE6AOJS|     798450889| 2015-08-31|
|R25IK0UAHWNB22|   34112894|B00COL0B7A|     777554234| 2015-08-31|
|R2A9IHKZMTMAL1|   20691979|B00QHLSKOE|     885228855| 2015-08-31|
| R5XVKTHL6SITI|   25983343|B00UNL2MUW|     164482798| 2015-08-31|
|R2QZZOSTDDY1IE|   35816068|B00RIC9JB4|     184834831| 2015-08-31|
|R38CUDCFPSNYTD|   10628020|B00HPMCO6O|     444378461| 2015-08

In [9]:
# Create the vine_table. DataFrame
vine_df = df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase']).drop_duplicates(subset=['review_id'])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R104OU974HV2CR|          5|            1|          1|   N|                Y|
|R105HWVSE9CJA3|          3|            6|          6|   N|                N|
|R105SFC97HDHHH|          5|            0|          0|   N|                Y|
|R1062I2AZSB0DS|          1|            0|          0|   N|                Y|
|R109TQYALV0SX8|          4|            0|          1|   N|                Y|
|R10AD5SLBCAFFB|          5|            0|          1|   N|                Y|
|R10APZTDYE3SQK|          5|            0|          0|   N|                Y|
|R10B61ONRINL6C|          5|            1|          3|   N|                Y|
|R10CCMYINZ2OQI|          4|            0|          0|   N|                Y|
|R10ENQ2XSAMEGC|          5|            0|          2|   N|     

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [10]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://challenge1.cbxkzogjn41l.us-east-2.rds.amazonaws.com:5432/challenge"
config = {"user":"postgres",
          "password":"<password>",
          "driver":"org.postgresql.Driver"}

In [11]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [12]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [13]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [14]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)